<a href="https://colab.research.google.com/github/sebug/hype-combiner/blob/main/LinkedInLearning/leprogramme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Challenge
Have a look at [Le Programme](https://leprogramme.ch/) - this is an events agenda for Geneva. We want to query it using natural language. The idea is to construct the search URL, and let you go there, but only if we constructed something valid. We can double check the combination between events returned and search query by filtering the full list as well - listing the events on the main page that match, and then finding a search clause that gives the most similar results.

I don't know whether this works yet, and will be exploring a lot.

The first step is to determine the current week of the year. We use this to get the events we will train on.

In [3]:
import datetime
current_week = datetime.date.today().isocalendar().week
current_year = datetime.date.today().isocalendar().year
current_week_url = 'https://leprogramme.ch/agenda-culturel-de-la-semaine/Vaud/' + str(current_year) + '/' + str(current_week)
current_week_url

'https://leprogramme.ch/agenda-culturel-de-la-semaine/Vaud/2024/19'

In [4]:
import urllib.request
programme_response = urllib.request.urlopen(current_week_url)
programme_content = programme_response.read()
print(programme_content)

b'\n<!DOCTYPE html>\n<html lang="en"><head>\n\n    <meta http-equiv="Content-Security-Policy" content="upgrade-insecure-requests">\n    <meta name="google-site-verification" content="VsCKkW7qEQybw6oaG_dJ869KxnLQpO5xvaw8SrdCE2g" />\n    <meta charset="utf-8"/>    <meta name="author" content="Lunatyk"/><meta name="viewport" content="width=device-width, initial-scale=1"/>    <meta name="description" content="Agenda concerts, th\xc3\xa9\xc3\xa2tre, danse dans le canton de Gen\xc3\xa8ve, semaine 32 - 2021"/>    <title>Agenda concerts, th\xc3\xa9\xc3\xa2tre, danse dans le canton de Gen\xc3\xa8ve, semaine 32 - 2021 </title>\n    <link href="/favicon.ico" type="image/x-icon" rel="icon"/><link href="/favicon.ico" type="image/x-icon" rel="shortcut icon"/>\n    \n\n\n    <link rel="apple-touch-icon" sizes="57x57" href="https://leprogramme.ch/img/favicon/apple-icon-57x57.png">\n    <link rel="apple-touch-icon" sizes="60x60" href="https://leprogramme.ch/img/favicon/apple-icon-60x60.png">\n    <link

We will need beautifulsoup to analyze the page content

In [5]:
!pip install beautifulsoup4

Let us get the events

In [12]:
from bs4 import BeautifulSoup
programme_soup = BeautifulSoup(programme_content)
spectacle_cards = [tag for tag in programme_soup.find_all('a', class_ = 'card-spectacle') ]
len(spectacle_cards)

48

and inspect the first card, to see how we can extract the relevant information. We may have to look at the parent to see the date we are talking about.

In [24]:
from datetime import datetime
spectacle = spectacle_cards[0]
date_string = spectacle.parent.parent.find('div', class_='spectacle-date').find('div', class_='day').get_text()

french_months = {
    'Janvier': '01',
    'Février': '02',
    'Mars': '03',
    'Avril': '04',
    'Mai': '05',
    'Juin': '06',
    'Juillet': '07',
    'Août': '08',
    'Septembre': '09',
    'Octobre': '10',
    'Novembre': '11',
    'Décembre': '12'
}

day, month_name, year = date_string.split()
month = french_months[month_name]
date_formatted = f"{day} {month} {year}"
date_object = datetime.strptime(date_formatted, "%d %m %Y")
date_object

datetime.datetime(2024, 5, 7, 0, 0)